In [1]:
from utilities import checkPII, translate, detect_language, retrieveFromKB, check_permission
import pprint, time
pp = pprint.PrettyPrinter(indent=2)

/home/ec2-user/anaconda3/envs/bhenv/lib/python3.10/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/bhenv/lib/python3.10/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/bhenv/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your install

In [2]:
queries = [\
              "My age is 61, what is the retirement age in India?",\
              "I did not receive login details for my new hire. what do i do?",\
             "Mi dirección es 1234 Richard Ln SE, MN 56789. ¿Cuándo recibiré mi recibo de sueldo?",\
            ]

In [3]:
time.time()

1713390999.4676273

In [10]:
query = queries[0]
kb_id = 'PUSBK6A7DJ'
for query in queries:
    start_time = time.time()
    ori_lang = detect_language(query)
    elapsed_time = time.time() - start_time
    print(f"Time taken for language detection: {elapsed_time} seconds")
    start_time = time.time()
    if ori_lang.lower() not in ['en', 'english']:
        query = translate(query,lang="English")
    elapsed_time = time.time() - start_time
    print(f"Time taken for language translation: {elapsed_time} seconds")
    start_time = time.time()
    PII_exists = await checkPII(query)
    elapsed_time = time.time() - start_time
    print(f"Time taken for PII check: {elapsed_time} seconds")
    print (query, ori_lang, PII_exists)
    start_time = time.time()
    response = retrieveFromKB(query, kb_id, 5)
    retrievalResults = response['retrievalResults']
    print([x['metadata']['Short description'] for x in retrievalResults])
    elapsed_time = time.time() - start_time
    print(f"Time taken for retrieval: {elapsed_time} seconds")
    start_time = time.time()
    print(check_permission(retrievalResults,'Candidate','India', ))


Parameter temperature does not exist for BedrockChat


Time taken for language detection: 0.3664720058441162 seconds
Time taken for language translation: 0.00023412704467773438 seconds


Parameter temperature does not exist for BedrockChat


Time taken for PII check: 3.210392951965332 seconds
My age is 61, what is the retirement age in India? English False
['Benefits at Glance_Ireland', 'Benefits at Glance_KR', 'Benefits at Glance_MY', 'Benefits at Glance - Egypt', 'Benefits at Glance_India']
Time taken for retrieval: 0.6450903415679932 seconds
['no', 'no', 'no', 'no', 'yes']


Parameter temperature does not exist for BedrockChat


...

Time taken for PII check: 3.661233425140381 seconds
I did not receive login details for my new hire. what do i do? English False
['How People Leaders Can Prepare for Their New Hires - Germany', 'Day 1 Logistics for Managers - Kuwait', 'Day 1 Logistics for Managers- UAE', 'Day 1 Logistics for Managers- Ireland', 'How People Leaders Can Prepare for Their New Hires - UAE']
Time taken for retrieval: 0.39232969284057617 seconds
['no', 'no', 'no', 'no', 'no']
Time taken for language detection: 0.24105072021484375 seconds


Parameter temperature does not exist for BedrockChat


Time taken for language translation: 0.6649444103240967 seconds


Parameter temperature does not exist for BedrockChat


Time taken for PII check: 3.887368679046631 seconds
My address is 1234 Richard Ln SE, MN 56789. When will I receive my paycheck? Spanish True
['Day 1 Other Info - Mexico (LE 0330, 0341, 0356, 0357, 0395)', 'Day 1 Other Info - Nigeria', 'Day 1 Other Info - Angola', 'Day 1 Other Info - Netherlands', 'Useful information for your first few days at Baker Hughes']
Time taken for retrieval: 0.5475080013275146 seconds
['no', 'no', 'no', 'no', 'no']


In [5]:
import boto3, json
s3 = boto3.client('s3')



bucket_name = 'genaihrpoc'
file_name = 'sample_output.txt'

s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(retrievalResults))


{'ResponseMetadata': {'RequestId': '8TNMFY0VS1BBC28M',
  'HostId': '4kFx2rz4voNkpBlwSnvYx7Dn642nS7grah1sC3qMdb9a2t3FPIpWmbf6iDIphHZT4O1jasUQ0co=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '4kFx2rz4voNkpBlwSnvYx7Dn642nS7grah1sC3qMdb9a2t3FPIpWmbf6iDIphHZT4O1jasUQ0co=',
   'x-amz-request-id': '8TNMFY0VS1BBC28M',
   'date': 'Wed, 17 Apr 2024 21:56:55 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"ef1329e516f854b1228cb8769d739328"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ef1329e516f854b1228cb8769d739328"',
 'ServerSideEncryption': 'AES256'}

In [9]:
retrievalResults[0]['metadata']['Short description']

'Day 1 Other Info - Mexico (LE 0330, 0341, 0356, 0357, 0395)'